In [1]:
import itertools
import pandas as pd
import numpy as np
import toytree
import toyplot
import arviz as az
import pymc3 as pm
from pymc3.distributions.dist_math import normal_lccdf, normal_lcdf

### Tree and dataframe setup

In [2]:
# generate a random tree
NSPECIES = 80
TREE = toytree.rtree.bdtree(
    ntips=NSPECIES,
    seed=444,
).mod.node_scale_root_height(1.0)

# node idxs that delimit several distinct clades on this tree
CLADES = [152, 153, 154, 155]

# draw and color the four major clades
TREE.draw(
    layout='d', 
    width=500,
    tip_labels=False,
    edge_colors=TREE.get_edge_values_mapped({
        j: toytree.colors[i] for i, j in enumerate(CLADES)
    }),
    scalebar=True,
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t84e7dec1d8774f14b1af394ea65d7961" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.0 0.5 1.0

In [3]:
# make group index (gidx)
crown_dict = {i: TREE.get_tip_labels(i) for i in CLADES}
gidx = np.zeros(TREE.ntips, dtype=int)
for tidx, tip in enumerate(TREE.get_tip_labels()):
    for cidx, clade in enumerate(crown_dict):
        if tip in crown_dict[clade]:
            gidx[tidx] = cidx
gidx

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [4]:
# True param values
𝛼_mean = 0.05
𝛼_std = 0.02
𝛽_mean = 1.5
𝛽_std = 0.05
𝜓_mean = 0.0
𝜓_std = 0.33
𝜎_std = 0.05

# 4 different clade effects on rate of RI (used for partial-pooling data)
𝜓_0_mean = 1.0
𝜓_0_std = 0.1
𝜓_1_mean = 0.5
𝜓_1_std = 0.05
𝜓_2_mean = -0.5
𝜓_2_std = 0.05
𝜓_3_mean = -1.0
𝜓_3_std = 0.1

In [5]:
# species dataframe
SPECIES_DATA = pd.DataFrame({
    "gidx": gidx,
    "b": np.random.normal(𝛽_mean, 𝛽_std, TREE.ntips),
    "psi": np.random.normal(𝜓_mean, 𝜓_std, TREE.ntips),
    "psi_x": np.concatenate([
        np.random.normal(𝜓_0_mean, 𝜓_0_std, len(gidx[gidx == 0])),
        np.random.normal(𝜓_1_mean, 𝜓_1_std, len(gidx[gidx == 1])),
        np.random.normal(𝜓_2_mean, 𝜓_2_std, len(gidx[gidx == 2])),
        np.random.normal(𝜓_3_mean, 𝜓_3_std, len(gidx[gidx == 3])),
    ]),
})
SPECIES_DATA.head()

,gidx,b,psi,psi_x
0,0,1.536410,0.229932,1.115651
1,0,1.552872,0.026556,0.904660
2,0,1.487890,-0.321123,1.075748
3,0,1.528174,-0.086356,1.094559
4,0,1.503244,0.444541,1.042192


### Generate crossing data

In [6]:
def get_dist(tree, idx0, idx1):
    "returns the genetic distance between two nodes on a tree"
    dist = tree.treenode.get_distance(
        tree.idx_dict[idx0], 
        tree.idx_dict[idx1],
    )
    return dist

In [7]:
# get all combinations of two sampled taxa
a, b = zip(*itertools.combinations(range(NSPECIES), 2))

# organize into DF and get genetic distance between pairs
DATA = pd.DataFrame({
    "sidx0": a,
    "sidx1": b,
    "dist": [(get_dist(TREE, i, j) / 2) for (i, j) in zip(a, b)],
})

DATA['b'] = np.random.normal(𝛽_mean, 𝛽_std, DATA.shape[0])
DATA['velo'] = (
    DATA['b']
    + SPECIES_DATA['psi'][DATA.sidx0].values
    + SPECIES_DATA['psi'][DATA.sidx1].values
)
DATA['velo_x'] = (
    DATA['b']
    + SPECIES_DATA['psi_x'][DATA.sidx0].values
    + SPECIES_DATA['psi_x'][DATA.sidx1].values
)
DATA['intercept'] = np.random.normal(𝛼_mean, 𝛼_std, DATA.shape[0])
DATA['error'] = np.random.normal(0.0, 𝜎_std, DATA.shape[0])

DATA['RI_pooled'] = DATA.b * DATA.dist ** 2 + DATA.intercept * DATA.dist + DATA.error
DATA['RI_unpooled'] = DATA.velo * DATA.dist ** 2 + DATA.intercept * DATA.dist + DATA.error
DATA['RI_partpooled'] = DATA.velo_x * DATA.dist ** 2 + DATA.intercept * DATA.dist + DATA.error

In [8]:
# censor values in range 0-1
DATA.loc[DATA['RI_pooled'] < 0, 'RI_pooled'] = 0
DATA.loc[DATA['RI_pooled'] > 1, 'RI_pooled'] = 1
DATA.loc[DATA['RI_unpooled'] < 0, 'RI_unpooled'] = 0
DATA.loc[DATA['RI_unpooled'] > 1, 'RI_unpooled'] = 1
DATA.loc[DATA['RI_partpooled'] < 0, 'RI_partpooled'] = 0
DATA.loc[DATA['RI_partpooled'] > 1, 'RI_partpooled'] = 1

In [9]:
DATA

,sidx0,sidx1,dist,b,velo,velo_x,intercept,error,RI_pooled,RI_unpooled,RI_partpooled
0,0,1,0.014093,1.569123,1.825611,3.589434,0.058843,0.003385,0.004526,0.004577,0.004927
1,0,2,0.093817,1.475324,1.384133,3.666723,0.060876,-0.006604,0.012093,0.011290,0.031381
2,0,3,0.232354,1.524398,1.667974,3.734608,0.063433,-0.013034,0.084005,0.091756,0.203331
3,0,4,0.232354,1.568007,2.242480,3.725850,0.080916,0.064646,0.168101,0.204515,0.284600
4,0,5,0.234463,1.587113,2.505924,3.761738,0.074592,-0.058734,0.046003,0.096513,0.165548
...,...,...,...,...,...,...,...,...,...,...,...
3155,76,78,0.057491,1.515498,2.497094,-0.487928,0.030432,-0.105419,0.000000,0.000000,0.000000
3156,76,79,0.194995,1.430819,1.861327,-0.579221,0.045074,-0.024966,0.038227,0.054597,0.000000
3157,77,78,0.057491,1.455528,2.218625,-0.729714,0.018721,0.122620,0.128507,0.131029,0.121284
3158,77,79,0.194995,1.432789,1.644798,-0.759066,0.117312,0.051167,0.128521,0.136582,0.045180


In [10]:
NSAMPLES = 1000
SAMPLE = DATA.sample(NSAMPLES).copy().reset_index(drop=True)
SAMPLE.head()

,sidx0,sidx1,dist,b,velo,velo_x,intercept,error,RI_pooled,RI_unpooled,RI_partpooled
0,1,26,0.523417,1.570259,1.690114,3.555288,0.037579,-0.025917,0.423948,0.456784,0.967777
1,33,53,0.555803,1.505988,0.773893,3.082311,0.001843,-0.020690,0.445560,0.219403,0.932514
2,34,63,1.000000,1.564016,1.027060,1.963146,0.065470,-0.015290,1.000000,1.000000,1.000000
3,13,67,1.000000,1.458742,1.518098,1.991522,0.042036,0.031772,1.000000,1.000000,1.000000
4,18,45,0.541446,1.532731,1.904726,3.538284,0.025705,0.091338,0.554598,0.663654,1.000000


### Visualize data

In [11]:
canvas = toyplot.Canvas(width=350, height=300)
axes = canvas.cartesian(
    label="Pooled data",
    xlabel="Genetic dist.",
    ylabel="Reprod. incomp.",
)

# points are jittered on x-axis for visibility
axes.scatterplot(
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    size=8,
    opacity=0.25,
    color=SAMPLE.loc[:, 'RI_pooled'].values,
    mstyle={"stroke": "black"},
);
axes.x.ticks.show = True
axes.y.ticks.show = True

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t4cc0aea9679a427397c8a53e8b22c8bb" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(5%,24.7%,46.2%);fill-opacity:1.0;opacity:0.25;stroke:rgb(0%,0%,0%);stroke-opacity:1.0

In [12]:
canvas = toyplot.Canvas(width=350, height=300)
axes = canvas.cartesian(
    label="Unpooled data",
    xlabel="Genetic dist.",
    ylabel="Reprod. incomp.",
)

# points are jittered on x-axis for visibility
axes.scatterplot(
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    size=8,
    opacity=0.25,
    color=SAMPLE.loc[:, 'RI_unpooled'].values,
    mstyle={"stroke": "black"},
);
axes.x.ticks.show = True
axes.y.ticks.show = True

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tf507e2036cb848e68fab7c47c0d842ea" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40.4%,0%,12.2%);fill-opacity:1.0;opacity:0.25;stroke:rgb(0%,0%,0%);s

In [13]:
canvas = toyplot.Canvas(width=350, height=300)
axes = canvas.cartesian(
    label="Partpooled data",
    xlabel="Genetic dist.",
    ylabel="Reprod. incomp.",
)

# points are jittered on x-axis for visibility
axes.scatterplot(
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    size=8,
    opacity=0.25,
    color=SAMPLE.loc[:, 'RI_partpooled'].values,
    mstyle={"stroke": "black"},
);
axes.x.ticks.show = True
axes.y.ticks.show = True

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t58961e29f59f4c9a8377415c4c261b26" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toypl

### Funcrions to plot results

In [14]:
def toytrace(trace, var_names, titles):
    """
    Plot posterior trace with toyplot
    """
    nvars = len(var_names)
    
    # setup canvase
    canvas = toyplot.Canvas(width=500, height=200 * nvars)
    
    # store axes
    axes = []
    
    # iter over params
    for pidx, param in enumerate(var_names):
        
        # get param posterior
        posterior = trace.get_values(param)
        
        # setup axes 
        ax = canvas.cartesian(grid=(nvars, 1, pidx))
        ax.y.show = False
        ax.x.spine.style = {"stroke-width": 1.5}
        ax.x.ticks.labels.style = {"font-size": "12px"}
        ax.x.ticks.show = True
        ax.x.label.text = f"param='{titles[pidx]}'"        
        
        # iterate over shape of param
        for idx in range(posterior.shape[1]):
            mags, bins = np.histogram(posterior[:, idx], bins=100)
            ax.plot(bins[1:], mags, stroke_width=2, opacity=0.6)
        axes.append(ax)
    return canvas, axes

In [15]:
import scipy.stats as stats

def draw_velocity_dists(trace, baseline=0.15):
    """
    Draw the clade velocities as gaussians
    """
    canvas = toyplot.Canvas(width=350, height=300)
    axes = canvas.cartesian(xlabel="Relative velocity of reproductive isolation")
    marks = []
    base = 0
    for i in range(trace['𝜓_mean'].shape[1]):
        
        loc = trace['𝜓_mean'][:, i].mean()
        scale = trace['𝜓_std'][:, i].mean()
        interval = stats.norm.interval(0.995, loc, scale)
        points = np.linspace(interval[0], interval[1], 100)
        mark = axes.fill(
            points, 
            stats.norm.pdf(points, loc=loc, scale=scale), 
            style={
                "fill-opacity": 0.45,
                "stroke": 'black',
                "stroke-opacity": 1.0,
                "stroke-width": 1,
            },
            baseline=np.repeat(base, 100),
        )
        marks.append(mark)
        axes.hlines(base, style={"stroke-dasharray": "5,5", 'stroke-width': 1})
        base += baseline
        
    axes.y.show = False
    axes.x.ticks.locator = toyplot.locator.Extended(only_inside=True)
    axes.x.ticks.show = True
    return canvas, axes, marks

### Define models

In [19]:
def censored_pooled_regression(x, y, **kwargs):
    
    # data pre-processing
    lower_censored = y[y <= 0].index
    _x_lc = x[lower_censored].values
    _y_lc = y[lower_censored].values

    upper_censored = y[y >= 1].index
    _x_uc = x[upper_censored].values
    _y_uc = y[upper_censored].values

    uncensored = (y > 0) & (y < 1)
    _x = x[uncensored].values
    _y = y[uncensored].values
    
    # define model
    with pm.Model() as model:  

        # parameters and error
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        𝜎 = pm.HalfNormal('𝜎', 5.0, shape=1)

        # linear model prediction
        ri = (𝛽 * _x ** 2) + 𝛼 * _x

        # data likelihood (normal distributed errors)
        y = pm.Normal("y", mu=ri, sigma=𝜎, observed=_y)

        # density of censored data
        if sum(lower_censored):
            lcensored = pm.Potential(
                "lower_censored", 
                normal_lcdf((𝛽 * _x_lc ** 2) + 𝛼 * _x_lc, 𝜎, _y_lc),
            )
        if sum(upper_censored):
            ucensored = pm.Potential(
                "upper_censored",
                normal_lccdf((𝛽 * _x_uc ** 2) + 𝛼 * _x_uc, 𝜎, _y_uc),
            )

        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]
    
        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [20]:
def censored_unpooled_noncentered_regression(x, y, idx0, idx1, **kwargs):
    
    # data pre-processing
    lower_censored = y[y <= 0].index
    _x_lc = x[lower_censored].values
    _y_lc = y[lower_censored].values

    upper_censored = y[y >= 1].index
    _x_uc = x[upper_censored].values
    _y_uc = y[upper_censored].values

    uncensored = (y > 0) & (y < 1)
    _x = x[uncensored].values
    _y = y[uncensored].values
    
    # define model
    with pm.Model() as model:
        
        # censored indexers
        sidx0 = pm.Data("spp_idx0", idx0.values[uncensored])
        sidx1 = pm.Data("spp_idx1", idx1.values[uncensored])
        sidx0_u = pm.Data("sidx0_u", idx0.values[upper_censored])
        sidx1_u = pm.Data("sidx1_u", idx1.values[upper_censored])
        sidx0_l = pm.Data("sidx0_l", idx0.values[lower_censored])
        sidx1_l = pm.Data("sidx1_l", idx1.values[lower_censored])

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=1)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=1)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean + 𝜓_std * 𝜓_offset)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝜎 = pm.HalfNormal('𝜎', 5., shape=1)
        
        # linear model prediction
        ri = ((𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * _x ** 2) + 𝛼 * _x

        # data likelihood (normal distributed errors)
        y = pm.Normal("y", mu=ri, sigma=𝜎, observed=_y)

        # density of censored data
        if sum(lower_censored):
            lcensored = pm.Potential(
                "lower_censored", 
                normal_lcdf(((𝛽 + 𝜓[sidx0_l] + 𝜓[sidx1_l]) * _x_lc ** 2) + 𝛼 * _x_lc, 𝜎, _y_lc),
            )
        if sum(upper_censored):
            ucensored = pm.Potential(
                "upper_censored",
                normal_lccdf(((𝛽 + 𝜓[sidx0_u] + 𝜓[sidx1_u]) * _x_uc ** 2) + 𝛼 * _x_uc, 𝜎, _y_uc),
            )

        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]

        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [21]:
def censored_partpooled_noncentered_regression(x, y, idx0, idx1, gidx, **kwargs):
    
    # data pre-processing
    lower_censored = y[y <= 0].index
    _x_lc = x[lower_censored].values
    _y_lc = y[lower_censored].values

    upper_censored = y[y >= 1].index
    _x_uc = x[upper_censored].values
    _y_uc = y[upper_censored].values

    uncensored = (y > 0) & (y < 1)
    _x = x[uncensored].values
    _y = y[uncensored].values
    
    # define model
    with pm.Model() as model:
        
        # censored indexers
        sidx0 = pm.Data("spp_idx0", idx0.values[uncensored])
        sidx1 = pm.Data("spp_idx1", idx1.values[uncensored])
        sidx0_u = pm.Data("sidx0_u", idx0.values[upper_censored])
        sidx1_u = pm.Data("sidx1_u", idx1.values[upper_censored])
        sidx0_l = pm.Data("sidx0_l", idx0.values[lower_censored])
        sidx1_l = pm.Data("sidx1_l", idx1.values[lower_censored])
        gidx = pm.Data("gidx", gidx)

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=4)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=4)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean[gidx] + 𝜓_std[gidx] * 𝜓_offset)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝜎 = pm.HalfNormal('𝜎', 5., shape=1)
        
        # linear model prediction
        ri = ((𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * _x ** 2) + 𝛼 * _x

        # data likelihood (normal distributed errors)
        y = pm.Normal("y", mu=ri, sigma=𝜎, observed=_y)

        # density of censored data
        if sum(lower_censored):
            lcensored = pm.Potential(
                "lower_censored", 
                normal_lcdf(((𝛽 + 𝜓[sidx0_l] + 𝜓[sidx1_l]) * _x_lc ** 2) + 𝛼 * _x_lc, 𝜎, _y_lc),
            )
        if sum(upper_censored):
            ucensored = pm.Potential(
                "upper_censored",
                normal_lccdf(((𝛽 + 𝜓[sidx0_u] + 𝜓[sidx1_u]) * _x_uc ** 2) + 𝛼 * _x_uc, 𝜎, _y_uc),
            )


        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]

        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

### Run models

In [22]:
# MCMC sampler kwargs
sample_kwargs = dict(
    tune=4000,
    draws=4000,
    target_accept=0.95,
    return_inferencedata=False,
    progressbar=True,
)

In [23]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_sub = censored_pooled_regression(*model_args[:2], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝜎, 𝛽, 𝛼]


Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 142 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [24]:
pooled_sub['stats']

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
𝛼[0],0.061,0.032,0.004,0.124,0.001,0.000,2960.0,2899.0,2957.0,3533.0,1.0
𝛽[0],1.466,0.058,1.352,1.571,0.001,0.001,2968.0,2968.0,2964.0,3592.0,1.0
𝜎[0],0.052,0.001,0.049,0.054,0.000,0.000,4737.0,4729.0,4728.0,4841.0,1.0


In [25]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_sub = censored_unpooled_noncentered_regression(*model_args[:4], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝜎, 𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 6373 seconds.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 25 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8384200855444406, but should be close to 0.95. Try to increase the number of tuning steps.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


In [26]:
unpooled_sub['stats']

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
𝜓_mean[0],0.459,3.606,-6.313,6.943,0.245,0.174,216.0,216.0,218.0,603.0,1.00
𝜓_offset[0],0.421,0.153,0.117,0.689,0.039,0.028,16.0,16.0,16.0,258.0,1.18
𝜓_offset[1],0.125,0.152,-0.151,0.415,0.042,0.031,13.0,13.0,13.0,70.0,1.23
𝜓_offset[2],-0.645,0.130,-0.889,-0.414,0.039,0.030,11.0,10.0,11.0,47.0,1.28
𝜓_offset[3],-0.353,0.165,-0.659,-0.050,0.031,0.022,29.0,29.0,29.0,218.0,1.11
...,...,...,...,...,...,...,...,...,...,...,...
𝜓[76],0.848,3.606,-5.989,7.278,0.245,0.173,217.0,217.0,219.0,598.0,1.00
𝜓[77],0.733,3.607,-6.129,7.142,0.245,0.173,217.0,217.0,219.0,599.0,1.00
𝜓[78],1.050,3.606,-5.730,7.524,0.245,0.174,216.0,216.0,218.0,598.0,1.00
𝜓[79],0.517,3.603,-6.027,7.208,0.245,0.173,216.0,216.0,218.0,608.0,1.00


In [27]:
toytrace(unpooled_sub['trace'], ['𝜓_mean', '𝜓_offset', '𝜓'], ['psi-mean', 'psi-offset', 'psi-spp']);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t7685ef4b09e04ec5976e5d2cf3b6669d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 600.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0 149.4475138121547 L 54.040404040404056 150.0 L 58.08080808080808 149.72375690607734 L 62.12121212121214 150.0 L 66.16161616161617 150.0 L 70.20202020202021 150.0 L 74.24242424242426 149.4475138121547 L 78.28282828282829 149.4475138121547 L 82.32323232323235 148.89502762430942 L 86.36363636363637 149.17127071823202 L 90.4040404040404 149.72375690607734 L 94.44444444444446 148.61878453038673 L 98.4848484848485 147.79005524861878 L 102.52525252525254 147.79005524861878 L 106.56565656565658 148.61878453038673 L 110.60606060606061 148.06629834254144 L 114.64646464646466 146.13259668508286 L 118.68686868686872 142.26519337016575 L 122.72727272727273 145.58011049723757 L 126.76767676767678 145.85635359116023 L 130.80808080808083 140.05524861878456 L 134.84848484848487 139.5027624309392 L 138.8888888888889 136.7403314917127 L 142.92929292929296 135.91160220994473 L 146.969696969697 133.4254143646409 L 151.010101010101 128.7292817679558 L 155.05050505050505 127.34806629834254 L 159.09090909090912 123.48066298342542 L 163.13131313131316 122.09944751381215 L 167.17171717171718 117.67955801104972 L 171.21212121212122 112.43093922651934 L 175.25252525252523 112.707182320442 L 179.2929292929293 107.73480662983425 L 183.33333333333337 106.62983425414366 L 187.37373737373736 104.69613259668509 L 191.41414141414145 96.96132596685082 L 195.45454545454547 95.02762430939228 L 199.4949494949495 91.71270718232044 L 203.53535353535355 85.6353591160221 L 207.57575757575756 81.76795580110497 L 211.61616161616163 77.90055248618785 L 215.6565656565657 70.44198895027625 L 219.6969696969697 79.8342541436464 L 223.73737373737376 74.30939226519337 L 227.77777777777777 75.69060773480663 L 231.81818181818184 76.24309392265194 L 235.85858585858588 75.41436464088397 L 239.89898989898992 75.96685082872929 L 243.93939393939397 50.0 L 247.979797979798 59.392265193370164 L 252.02020202020202 72.37569060773481 L 256.06060606060606 58.83977900552486 L 260.1010101010101 56.353591160220994 L 264.14141414141415 61.87845303867404 L 268.1818181818182 57.73480662983426 L 272.22222222222223 56.62983425414365 L 276.26262626262627 59.392265193370164 L 280.3030303030303 64.9171270718232 L 284.34343434343435 64.64088397790056 L 288.3838383838384 72.37569060773481 L 292.42424242424244 82.59668508287292 L 296.4646464646464 69.06077348066299 L 300.5050505050505 76.79558011049724 L 304.54545454545456 91.9889502762431 L 308.5858585858586 86.1878453038674 L 312.6262626262626 86.1878453038674 L 316.66666666666674 90.88397790055248 L 320.7070707070707 97.79005524861877 L 324.7474747474747 106.90607734806629 L 328.7878787878788 100.27624309392264 L 332.82828282828285 107.18232044198894 L 336.8686868686869 112.707182320442 L 340.90909090909093 117.40331491712709 L 344.949494949495 125.41436464088399 L 348.989898989899 122.9281767955801 L 353.030303030303 126.79558011049724 L 357.0707070707071 130.66298342541435 L 361.1111111111111 130.11049723756906 L 365.1515151515152 134.80662983425415 L 369.1919191919192 138.95027624309392 L 373.2323232323232 140.88397790055248 L 377.27272727272725 140.88397790055248 L 381.3131313131314 143.646408839779 L 385.3535353535354 143.92265193370164 L 389.3939393939394 145.02762430939228 L 393.4343434343434 145.3038674033149 L 397.4747474747475 148.34254143646407 L 401.51515151515156 148.61878453038673 L 405.55555555555554 147.2375690607735 L 409.5959595959596 148.06629834254144 L 413.6363636363636 148.3425

In [28]:
# show plot of TRUE vs. ESTIMATED rates
c, a, m = toyplot.scatterplot(
    unpooled_sub['trace']['𝜓'].mean(axis=0),         # estimated
    SPECIES_DATA['psi'],                             # true
    width=400,
    height=250,
    xlabel="ESTIMATED species velocity",
    ylabel="TRUE species velocity",
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="ta4e90c8215d24347ac7659f9b4f0b7c9" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 250.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> -0.5 0.0 0.5 1.0 ESTIMATED species velocity -1.0 -0.5 0.0 0.5 TRUE species velocity

In [29]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_sub = censored_partpooled_noncentered_regression(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝜎, 𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 4_000 tune and 4_000 draw iterations (16_000 + 16_000 draws total) took 7954 seconds.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


In [30]:
toytrace(partpooled_sub['trace'], ['𝜓_mean', '𝜓_offset', '𝜓'], ['psi-mean', 'psi-offset', 'psi-spp']);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t3c5ae23d9a6541aeb8983199a5cdf546" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 600.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 127.76964166985766 149.7275204359673 L 130.94346157839584 149.4550408719346 L 134.11728148693405 149.7275204359673 L 137.29110139547223 149.7275204359673 L 140.46492130401043 149.18256130790192 L 143.6387412125486 148.9100817438692 L 146.8125611210868 148.3651226158038 L 149.98638102962497 149.18256130790192 L 153.16020093816314 148.9100817438692 L 156.33402084670138 146.73024523160763 L 159.50784075523956 147.00272479564032 L 162.68166066377773 146.73024523160763 L 165.85548057231594 145.91280653950955 L 169.02930048085412 143.73297002724794 L 172.2031203893923 143.46049046321528 L 175.37694029793047 144.27792915531336 L 178.55076020646868 138.28337874659402 L 181.7245801150069 139.3732970027248 L 184.89840002354507 135.28610354223434 L 188.07221993208327 140.1907356948229 L 191.24603984062145 135.01362397820162 L 194.41985974915963 131.74386920980925 L 197.5936796576978 129.56403269754767 L 200.767499566236 128.7465940054496 L 203.9413194747742 124.11444141689374 L 207.11513938331237 122.75204359673025 L 210.28895929185057 121.11716621253404 L 213.4627792003888 121.93460490463215 L 216.63659910892696 115.39509536784742 L 219.81041901746514 108.85558583106268 L 222.98423892600331 102.58855585831063 L 226.15805883454152 102.86103542234333 L 229.33187874307973 98.22888283378748 L 232.5056986516179 88.96457765667574 L 235.6795185601561 87.87465940054496 L 238.8533384686943 81.88010899182561 L 242.02715837723247 72.6158038147139 L 245.20097828577067 76.43051771117166 L 248.37479819430882 74.25068119891009 L 251.54861810284706 67.71117166212534 L 254.72243801138524 64.16893732970027 L 257.8962579199234 65.53133514986376 L 261.0700778284616 65.25885558583107 L 264.24389773699977 50.2724795640327 L 267.41771764553795 62.53405994550408 L 270.5915375540762 56.26702997275205 L 273.76535746261436 60.89918256130791 L 276.93917737115254 50.0 L 280.1129972796908 66.34877384196186 L 283.28681718822895 56.53950953678474 L 286.4606370967671 56.26702997275205 L 289.6344570053053 57.629427792915536 L 292.80827691384354 63.35149863760218 L 295.9820968223817 60.3542234332425 L 299.1559167309199 75.06811989100818 L 302.3297366394581 75.06811989100818 L 305.50355654799625 79.42779291553134 L 308.67737645653443 85.96730245231608 L 311.8511963650726 88.41961852861036 L 315.02501627361085 86.78474114441417 L 318.198836182149 101.77111716621252 L 321.3726560906872 101.49863760217983 L 324.5464759992253 104.22343324250681 L 327.72029590776356 105.5858310626703 L 330.89411581630173 110.49046321525887 L 334.06793572483997 114.85013623978202 L 337.2417556333782 117.0299727520436 L 340.4155755419163 116.2125340599455 L 343.5893954504545 115.94005449591279 L 346.76321535899274 118.11989100817439 L 349.93703526753086 125.20435967302453 L 353.1108551760691 124.93188010899183 L 356.2846750846073 129.29155313351498 L 359.45849499314545 124.11444141689374 L 362.6323149016837 130.1089918256131 L 365.8061348102218 135.01362397820162 L 368.97995471876 137.7384196185286 L 372.15377462729816 140.4632152588556 L 375.32759453583645 143.73297002724794 L 378.50141444437463 144.00544959128067 L 381.6752343529128 144.27792915531336 L 384.849054261451 145.36784741144413 L 388.0228741699891 144.27792915531336 L 391.19669407852734 144.82288828337875 L 394.3705139870656 144.00544959128067 L 397.54433389560376 147.27520435967304 L 400.718153804142 146.73024523160763 L 403.8919737126801 149.4550408719346 L 407.0657936212183 147.8201

In [31]:
# show plot of TRUE vs. ESTIMATED rates
c, a, m = toyplot.scatterplot(
    partpooled_sub['trace']['𝜓'].mean(axis=0),         # estimated
    SPECIES_DATA['psi_x'],                             # true
    width=400,
    height=250,
    xlabel="ESTIMATED species velocity",
    ylabel="TRUE species velocity",
    color=[toyplot.color.Palette()[i] for i in SPECIES_DATA.gidx],
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t2d7eff63bf7e473ba760b595a8a3cb78" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 250.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> -1.0 -0.5 0.0 0.5 1.0 1.5 ESTIMATED species velocity -1 0 1 TRUE species velocity

In [32]:
draw_velocity_dists(partpooled_sub['trace'], baseline = 5);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t52bb52dba36b4cd2b8cb68f82ef46753" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="235.78697132919737,250.0 236.4355877804176,250.0 237.08420423163784,250.0 237.73282068285806,250.0 238.38143713407828,250.0 239.0300535852985,250.0 239.67867003651875,250.0 240.32728648773897,250.0 240.9759029389592,250.0 241.62451939017947,250.0 242.2731358413997,250.0 242.92175229261989,250.0 243.5703687438401,250.0 244.21898519506033,250.0 244.8676016462806,250.0 245.51621809750083,250.0 246.16483454872105,250.0 246.81345099994127,250.0 247.4620674511615,250.0 248.1106839023817,250.0 248.7593003536019,250.0 249.40791680482212,250.0 250.05653325604237,250.0 250.70514970726268,250.0 251.3537661584829,250.0 252.0023826097031,250.0 252.6509990609233,250.0 253.29961551214353,250.0 253.94823196336375,250.0 254.596848414584,250.0 255.24546486580422,250.0 255.89408131702447,250.0 256.5426977682447,250.0 257.1913142194649,250.0 257.83993067068513,250.0 258.48854712190536,250.0 259.13716357312563,250.0 259.78578002434585,250.0 260.4343964755661,250.0 261.0830129267863,250.0 261.7316293780065,250.0 262.38024582922674,250.0 263.02886228044696,250.0 263.6774787316672,250.0 264.32609518288746,250.0 264.9747116341077,250.0 265.6233280853279,250.0 266.2719445365481,250.0 266.92056098776834,250.0 267.56917743898856,250.0 268.2177938902088,250.0 268.866410341429,250.0 269.5150267926493,250.0 270.1636432438695,250.0 270.8122596950897,250.0 271.46087614630994,250.0 272.10949259753016,250.0 272.7581090487504,250.0 273.40672549997066,250.0 274.0553419511909,250.0 274.7039584024111,250.0 275.3525748536313,250.0 276.00119130485155,250.0 276.64980775607177,250.0 277.298424207292,250.0 277.9470406585122,250.0 278.5956571097325,250.0 279.24427356095276,250.0 279.892890012173,250.0 280.5415064633932,250.0 281.19012291461337,250.0 281.8387393658336,250.0 282.4873558170538,250.0 283.1359722682741,250.0 283.7845887194943,250.0 284.43320517071453,250.0 285.08182162193475,250.0 285.73043807315497,250.0 286.3790545243752,250.0 287.0276709755954,250.0 287.6762874268157,250.0 288.3249038780359,250.0 288.97352032925613,250.0 289.62213678047635,250.0 290.2707532316966,250.0 290.9193696829168,250.0 291.567986134137,250.0 292.21660258535724,250.0 292.86521903657746,250.0 293.51383548779773,250.0 294.16245193901796,250.0 294.8110683902382,250.0 295.4596848414584,250.0 296.1083012926786,250.0 296.7569177438988,250.0 297.4055341951191,250.0 298.05415064633934,250.0 298.70276709755956,250.0 299.3513835487798,250.0 300.0,250.0 300.0,249.49312740451464 299.3513835487798,249.4066215602716 298.70276709755956,249.307582361655 298.05415064633934,249.19460692400824 297.4055341951191,249.0662060194107 296.7569177438988,248.9208106330794 296.1083012926786,248.75678095529423 295.4596848414584,248.57241801015135 294.8110683902382,248.36597809044795 294.16245193901796,248.13569012636978 293.51383548779773,247.87977606418983 292.86521903657746,247.59647427002457 292.21660258535724,247.28406590328737 291.567986134137,246.94090412569258 290.9193696829168,246.5654459257518 290.2707532316966,246.15628624733912 289.62213678047635,245.71219401617662 288.97352032925613,245.2321495624606 288.3249038780359,244.71538284413762 287.6762874268157,244.16141178662295 287.0276709755954,243.57007997432612 286.3790545243752,242.94159286059434 285.73043807315497,242.27655160896953 285.08182162193475,241.5759836432104 284.43320517071453,240.84136896930346 283.7845887194943,240.074661342216 283.1359722682741,239.2783033854117 282.4873558170538,238

### Assess model fit

In [33]:
def rmse(predictions, targets):
    differences = predictions - targets                       #the DIFFERENCEs.
    differences_squared = differences ** 2                    #the SQUAREs of ^
    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^
    rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^
    return rmse_val                                           #get the ^

In [34]:
def aicm(mean, variance):
    return 2*mean - 2*variance

In [35]:
rmse(unpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi'])

0.48677400638369445

In [36]:
rmse(partpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi_x'])

0.22755457245620744

In [37]:
aicm(unpooled_sub['trace']['𝜓'].mean(), unpooled_sub['trace']['𝜓'].var())

-25.29491117881976

In [38]:
aicm(partpooled_sub['trace']['𝜓'].mean(), partpooled_sub['trace']['𝜓'].var())

-10.081659924536705